<a href="https://colab.research.google.com/github/Shashank-Ojha/WordFormer/blob/main/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
%load_ext autoreload
%autoreload 2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [10]:
import torch
from utils.attention_block import SelfAttention

# Set a seed for reproducibility
torch.manual_seed(0)



embed_dim = 6
kq_dim = 5
v_dim = 10


attention_block = SelfAttention(embed_dim, kq_dim, v_dim)

num_tokens = 4
X = torch.rand(num_tokens, embed_dim)
context_matrix = attention_block(X)

In [12]:
print(context_matrix)
# Should be (num_tokens, v_dim)
print(context_matrix.shape)

tensor([[2.4716, 2.4678, 3.1775, 1.9464, 1.9135, 2.3258, 2.3451, 0.9887, 2.1647,
         2.0355],
        [2.4012, 2.4032, 3.0854, 1.8844, 1.8565, 2.2656, 2.2799, 0.9622, 2.1139,
         1.9821],
        [2.5718, 2.5559, 3.3045, 2.0339, 1.9943, 2.4080, 2.4402, 1.0257, 2.2363,
         2.1122],
        [2.6694, 2.6405, 3.4262, 2.1195, 2.0721, 2.4872, 2.5324, 1.0623, 2.3059,
         2.1868]], grad_fn=<MmBackward0>)
torch.Size([4, 10])
